In [1]:
from IPython.display import Markdown, Latex
from datasets import load_dataset
import numpy as np
import pandas as pd

In [2]:
methods = [
    "danjacobellis/MUSDB_opus_12kbps",
    "danjacobellis/MUSDB_Stereo_Li_108c_test2",
    "danjacobellis/MUSDB_encodec_12kbps",
    "danjacobellis/MUSDB_Stereo_Li_27c_test2",
    "danjacobellis/MUSDB_stable_audio_fp16"
]

dimensionality_reduction = [
    1,
    512/108,
    5,
    512/27,
    64,
]

names = [
    'Opus',
    'WalloC',
    'EnCodec',
    'WalloC',
    'Stable Audio',
]

metrics = [
    'PSNR',
    'SSDR',
    'SRDR',
]

columns = [
    'Name',
    'Dimensionality Reduction',
    'Compression Ratio',
    'Encode',
    'Decode',
] + metrics + ['CDPAM']

In [3]:
data_rows = []

for i_method, method in enumerate(methods):
    ds = load_dataset(method, split='validation')
    row = [names[i_method], dimensionality_reduction[i_method]]
    bps_mean = np.mean(ds['bps'])
    compression_ratio = 16 / bps_mean
    encode_MS_per_second = 2*np.mean(np.array(ds['L'])/np.array(ds['cpu_encode_time'])/1e6)
    decode_MS_per_second = 2*np.mean(np.array(ds['L'])/np.array(ds['cpu_decode_time'])/1e6)
    row.append(compression_ratio)
    row.append(encode_MS_per_second)
    row.append(decode_MS_per_second)
    for metric in metrics:
        mean = np.mean(ds[metric])
        row.append(mean)
    CDPAM_dB = np.median(-10*np.log10(ds['CDPAM']))
    row.append(CDPAM_dB)
    data_rows.append(row)

df = pd.DataFrame(data_rows, columns=columns)

df_rounded = df.round(4)
md_table = df_rounded.to_markdown(index=False)
display(Markdown(md_table))

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

...

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

| Name         |   Dimensionality Reduction |   Compression Ratio |   Encode |   Decode |    PSNR |    SSDR |    SRDR |   CDPAM |
|:-------------|---------------------------:|--------------------:|---------:|---------:|--------:|--------:|--------:|--------:|
| Opus         |                     1      |            118.542  |  11.5    | 101.783  | 30.3705 | 16.7345 |  5.0343 | 40.3767 |
| WalloC       |                     4.7407 |             21.3065 |  77.7947 |  11.1713 | 39.0242 | 33.2625 | 13.8961 | 41.0717 |
| EnCodec      |                     5      |            114.131  |   2.7511 |   3.0337 | 31.9516 | 22.6949 |  6.6919 | 47.3817 |
| WalloC       |                    18.963  |             76.2823 | 121.26   |  12.1872 | 33.3161 | 22.523  |  8.061  | 36.641  |
| Stable Audio |                    64      |             64      |   0.3084 |   0.2998 | 28.4155 | 15.7432 |  2.0318 | 49.7486 |

In [4]:
df_rounded = df.round(10)
column_format = 'l' + 'rr' * len(metrics)
latex_table = df_rounded.to_latex(
    index=True,
    float_format="%.3g",
    column_format=column_format,
    multicolumn=True,
    multicolumn_format='c',
    bold_rows=True
)
print(latex_table)

\begin{tabular}{lrrrrrr}
\toprule
 & Name & Dimensionality Reduction & Compression Ratio & Encode & Decode & PSNR & SSDR & SRDR & CDPAM \\
\midrule
\textbf{0} & Opus & 1 & 119 & 11.5 & 102 & 30.4 & 16.7 & 5.03 & 40.4 \\
\textbf{1} & WalloC & 4.74 & 21.3 & 77.8 & 11.2 & 39 & 33.3 & 13.9 & 41.1 \\
\textbf{2} & EnCodec & 5 & 114 & 2.75 & 3.03 & 32 & 22.7 & 6.69 & 47.4 \\
\textbf{3} & WalloC & 19 & 76.3 & 121 & 12.2 & 33.3 & 22.5 & 8.06 & 36.6 \\
\textbf{4} & Stable Audio & 64 & 64 & 0.308 & 0.3 & 28.4 & 15.7 & 2.03 & 49.7 \\
\bottomrule
\end{tabular}

